In [ ]:
import numpy as np
import pandas as pd
import pickle
import time
import os
from typing import List, Tuple, Optional, Dict
from numpy.linalg import eigh
from sklearn.cluster import SpectralClustering
from itertools import combinations

# QOKit imports (keep these unchanged)
from qokit.portfolio_optimization import get_problem, portfolio_brute_force, get_sk_ini
from qokit.qaoa_circuit_portfolio import (
    get_qaoa_circuit,
    get_parameterized_qaoa_circuit,
    get_energy_expectation_sv,
)
from qokit.qaoa_objective_portfolio import get_qaoa_portfolio_objective
from qokit.utils import reverse_array_index_bit_order
from qiskit.quantum_info import Statevector
from qiskit import transpile
from qiskit_aer import Aer
import nlopt

In [ ]:

# Advanced decomposition functions from the research paper
def rmt_denoise(sigma, n_obs):
    """RMT denoising with stability improvements."""
    n = len(sigma)
    
    # Ensure symmetric
    sigma = (sigma + sigma.T) / 2
    
    # Add regularization
    sigma += np.eye(n) * 1e-8
    
    # Calculate correlation matrix
    std = np.sqrt(np.diag(sigma))
    std = np.maximum(std, 1e-10)
    corr = sigma / np.outer(std, std)
    np.fill_diagonal(corr, 1.0)
    
    # Eigenvalue decomposition
    vals, vecs = eigh(corr)
    vals, vecs = vals[::-1], vecs[:, ::-1]
    
    # Marchenko-Pastur bounds
    q = n / n_obs
    if q >= 1:
        q = 0.99
    
    lmin = (1 - np.sqrt(q)) ** 2
    lmax = (1 + np.sqrt(q)) ** 2
    
    # Clean eigenvalues
    clean_vals = vals.copy()
    noise_idx = (vals >= lmin) & (vals <= lmax)
    
    if np.any(noise_idx):
        avg_noise = np.mean(vals[noise_idx])
        clean_vals[noise_idx] = avg_noise
    
    # Remove market mode if dominant
    if clean_vals[0] > lmax * 2:
        clean_vals[0] = clean_vals[1]
    
    # Ensure positive
    clean_vals = np.maximum(clean_vals, 1e-8)
    
    # Reconstruct
    corr_clean = (vecs * clean_vals) @ vecs.T
    corr_clean = (corr_clean + corr_clean.T) / 2
    np.fill_diagonal(corr_clean, 1.0)
    
    return corr_clean * np.outer(std, std)


def spectral_clusters(corr, k, min_sz):
    """Spectral clustering on correlation matrix."""
    n = len(corr)
    
    # Affinity matrix
    aff = np.abs(corr) ** 2
    np.fill_diagonal(aff, 1.0)
    aff = (aff + aff.T) / 2
    
    # Clustering
    try:
        clustering = SpectralClustering(
            n_clusters=k,
            affinity='precomputed',
            random_state=42,
            n_init=10
        )
        labels = clustering.fit_predict(aff)
    except:
        # Fallback to random assignment
        labels = np.random.randint(0, k, size=n)
    
    clusters = [np.where(labels == i)[0].tolist() for i in range(k)]
    clusters = [c for c in clusters if len(c) > 0]
    
    # Merge small clusters
    i = 0
    while i < len(clusters):
        if len(clusters[i]) < min_sz and len(clusters) > 1:
            small = clusters.pop(i)
            # Add to random cluster
            if clusters:
                clusters[np.random.randint(len(clusters))].extend(small)
        else:
            i += 1
    
    # Ensure exactly k clusters if possible
    while len(clusters) < k and any(len(c) >= 2*min_sz for c in clusters):
        # Split largest
        largest_idx = max(range(len(clusters)), key=lambda i: len(clusters[i]))
        large = clusters[largest_idx]
        mid = len(large) // 2
        clusters[largest_idx] = large[:mid]
        clusters.append(large[mid:])
    
    return clusters[:k]


def partition_with_rmt(sigma, mu, size, n_obs=None):
    """
    Advanced partitioning using RMT denoising and spectral clustering.
    
    Parameters:
    -----------
    sigma : np.ndarray
        Covariance matrix
    mu : np.ndarray
        Expected returns vector
    size : int
        Target size for each partition
    n_obs : int
        Number of observations used to compute covariance (for RMT)
    
    Returns:
    --------
    partitioned_mus : List of expected return sub-vectors
    partitioned_sigmas : List of covariance sub-matrices
    mappings : List of indices for each partition
    """
    n = len(mu)
    
    # Step 1: Apply RMT denoising
    if n_obs is None:
        n_obs = 252 * 5  # Default to 5 years of daily data
    
    sigma_clean = rmt_denoise(sigma, n_obs)
    
    # Step 2: Calculate correlation for clustering
    std = np.sqrt(np.diag(sigma_clean))
    std = np.maximum(std, 1e-10)
    corr_clean = sigma_clean / np.outer(std, std)
    
    # Step 3: Determine number of clusters
    n_clusters = max(1, n // size)
    
    # Step 4: Perform spectral clustering
    clusters = spectral_clusters(corr_clean, n_clusters, min_sz=3)
    
    # Step 5: Extract submatrices and sub-vectors
    partitioned_sigmas = []
    partitioned_mus = []
    mappings = []
    
    for cluster in clusters:
        if len(cluster) > 0:
            cluster_array = np.array(cluster)
            sub_sigma = sigma_clean[np.ix_(cluster_array, cluster_array)]
            sub_mu = mu[cluster_array]
            
            partitioned_sigmas.append(sub_sigma)
            partitioned_mus.append(sub_mu)
            mappings.append(cluster)
    
    return partitioned_mus, partitioned_sigmas, mappings


def get_sub_problem_with_rebalancing(sigma, mu, K, q, seed=1, scale=1.0, 
                                    full_sigma=None, full_mu=None):
    """
    Create subproblem with risk rebalancing as in the paper.
    
    Includes the risk rebalancing formula from Eq. (15) in the paper.
    """
    po_problem = {
        "N": len(mu),
        "K": K,
        "q": q,
        "seed": seed,
        "means": mu,
        "cov": sigma,
        "pre": False
    }
    
    # Apply risk rebalancing if full problem info is provided
    if full_sigma is not None and full_mu is not None:
        # Risk rebalancing formula from the paper
        mu_ratio = np.linalg.norm(mu) / np.linalg.norm(full_mu)
        sigma_ratio = np.linalg.norm(sigma, 'fro') / np.linalg.norm(full_sigma, 'fro')
        q_rebalanced = q * (mu_ratio / sigma_ratio)
        po_problem["q"] = q_rebalanced
    
    # Apply scaling
    po_problem["scale"] = scale
    po_problem["means"] = scale * np.array(mu)
    po_problem["cov"] = scale * np.array(sigma)
    
    return po_problem


def map_qaoa_outputs_to_full_vector(outputs, mappings, n):
    """Map QAOA outputs back to full solution vector."""
    result = np.zeros(n, dtype=int)
    
    for bits, indices in zip(outputs, mappings):
        for bit, idx in zip(bits, indices):
            result[idx] = int(bit)
    
    return result


def minimize_nlopt(f, x0, p):
    """Minimize using NLopt BOBYQA algorithm."""
    def nlopt_wrapper(x, grad):
        return f(x).real
    
    opt = nlopt.opt(nlopt.LN_BOBYQA, 2 * p)
    opt.set_min_objective(nlopt_wrapper)
    opt.set_xtol_rel(1e-8)
    opt.set_ftol_rel(1e-8)
    opt.set_initial_step(0.01)
    
    xstar = opt.optimize(x0)
    minf = opt.last_optimum_value()
    
    return xstar, minf


def solve_qaoa_subproblem(po_problem, p=5, verbose=False):
    """
    Solve a single portfolio optimization subproblem using QAOA.
    
    Returns:
    - best_bitstring: The optimal asset selection as a bitstring
    - opt_params: The optimized QAOA parameters
    - ar: The approximation ratio for this subproblem
    """
    N = po_problem['N']
    K = po_problem['K']
    
    # Create QAOA objective function
    qaoa_obj = get_qaoa_portfolio_objective(
        po_problem=po_problem,
        p=p,
        ini='dicke',  # Dicke state initialization
        mixer='trotter_ring',  # Ring-XY mixer
        T=1,
        simulator='python'  # QOKit simulator
    )
    
    # Get initial parameters
    x0 = get_sk_ini(p=p)
    
    # Optimize QAOA parameters
    if verbose:
        print(f"  Optimizing {N}-asset subproblem with K={K}...")
    
    start_time = time.time()
    opt_params, opt_energy = minimize_nlopt(qaoa_obj, x0, p=p)
    opt_time = time.time() - start_time
    
    # Get the quantum state with optimal parameters
    gammas = opt_params[:p] / 2
    betas = opt_params[p:] / 2
    
    # Create circuit and get statevector
    qc = get_qaoa_circuit(po_problem, gammas=gammas, betas=betas, depth=p)
    sv = Statevector.from_instruction(qc)
    probs = sv.probabilities()
    
    # Find the most probable valid state (with K assets)
    best_prob = 0
    best_bitstring = None
    
    for i, prob in enumerate(probs):
        bitstring = format(i, f'0{N}b')
        if bitstring.count('1') == K and prob > best_prob:
            best_prob = prob
            best_bitstring = bitstring
    
    # Calculate approximation ratio
    best_classical = portfolio_brute_force(po_problem, return_bitstring=False)
    ar = (opt_energy - best_classical[1]) / (best_classical[0] - best_classical[1])
    
    if verbose:
        print(f"    Optimization time: {opt_time:.2f}s, AR: {ar:.3f}, Best prob: {best_prob:.3f}")
    
    return best_bitstring, opt_params, ar


def load_portfolio_data(pickle_file: str):
    """Load portfolio data from NetworkX graph pickle file."""
    with open(pickle_file, 'rb') as f:
        data = pickle.load(f)
    
    if hasattr(data, 'nodes') and hasattr(data, 'edges'):
        graph = data
        nodes = list(graph.nodes())
        n = len(nodes)
        
        # Extract returns and volatilities
        mu = np.zeros(n)
        volatilities = np.zeros(n)
        
        for i, node in enumerate(nodes):
            node_data = graph.nodes[node]
            mu[i] = node_data.get('annual_return', 0)
            volatilities[i] = node_data.get('annual_volatility', 0)
        
        # Build correlation matrix from edge weights
        correlation = np.eye(n)
        node_to_idx = {node: i for i, node in enumerate(nodes)}
        
        for node1, node2, edge_data in graph.edges(data=True):
            i = node_to_idx[node1]
            j = node_to_idx[node2]
            weight = edge_data.get('weight', 0)
            correlation[i, j] = weight
            correlation[j, i] = weight
        
        # Convert correlation to covariance
        sigma = np.outer(volatilities, volatilities) * correlation
        
        return sigma, mu
    
    raise ValueError("Expected NetworkX graph in pickle file")


def calculate_final_approximation_ratio(x: np.ndarray, sigma: np.ndarray, mu: np.ndarray, 
                                      K: int, q: float, n_samples: int = 1000) -> float:
    """
    Calculate approximation ratio for the final solution.
    
    For large problems where brute force is infeasible, we sample random solutions
    to estimate the worst case and use heuristics for the best case.
    """
    n = len(mu)
    current_obj = q * x.T @ sigma @ x - mu.T @ x
    
    # For small problems, use brute force
    if n <= 20:
        from itertools import combinations
        
        best_obj = float('inf')
        worst_obj = float('-inf')
        
        for selected in combinations(range(n), K):
            x_temp = np.zeros(n)
            x_temp[list(selected)] = 1
            obj = q * x_temp.T @ sigma @ x_temp - mu.T @ x_temp
            
            best_obj = min(best_obj, obj)
            worst_obj = max(worst_obj, obj)
    else:
        # For larger problems, use sampling and heuristics
        
        # Estimate worst case by sampling
        worst_obj = float('-inf')
        for _ in range(n_samples):
            indices = np.random.choice(n, K, replace=False)
            x_temp = np.zeros(n)
            x_temp[indices] = 1
            obj = q * x_temp.T @ sigma @ x_temp - mu.T @ x_temp
            worst_obj = max(worst_obj, obj)
        
        # Estimate best case using greedy heuristic
        # Start with assets that have best return/risk ratio
        scores = mu - q * np.diag(sigma)
        best_indices = np.argsort(scores)[-K:]
        x_best = np.zeros(n)
        x_best[best_indices] = 1
        best_obj = q * x_best.T @ sigma @ x_best - mu.T @ x_best
        
        # Refine with local search
        for _ in range(100):
            improved = False
            for i in range(n):
                if x_best[i] == 1:
                    for j in range(n):
                        if x_best[j] == 0:
                            # Try swapping
                            x_best[i] = 0
                            x_best[j] = 1
                            new_obj = q * x_best.T @ sigma @ x_best - mu.T @ x_best
                            if new_obj < best_obj:
                                best_obj = new_obj
                                improved = True
                                break
                            else:
                                # Revert
                                x_best[i] = 1
                                x_best[j] = 0
                    if improved:
                        break
            if not improved:
                break
    
    # Calculate approximation ratio
    if worst_obj != best_obj:
        ar = (worst_obj - current_obj) / (worst_obj - best_obj)
    else:
        ar = 1.0
    
    return max(0.0, min(1.0, ar))  # Clamp to [0, 1]


def main(pickle_file: str, partition_size: int = 15, q: float = 0.5, 
         K_total: int = 40, p: int = 5, scale: float = 1.0, 
         n_obs: Optional[int] = None, verbose: bool = True):
    """
    Main function using advanced partitioning with QAOA solving.
    
    Parameters:
    -----------
    pickle_file : str
        Path to the pickle file containing the portfolio graph
    partition_size : int
        Target size of each partition (default: 15)
    q : float
        Risk aversion parameter
    K_total : int
        Total number of assets to select across all partitions
    p : int
        QAOA circuit depth
    scale : float
        Scaling factor for the problem
    n_obs : int
        Number of observations for RMT denoising
    verbose : bool
        Whether to print progress information
    """
    
    # Step 1: Load the graph and extract portfolio data
    if verbose:
        print("Loading portfolio data from pickle file...")
    sigma, mu = load_portfolio_data(pickle_file)
    N = len(mu)
    
    if verbose:
        print(f"Loaded {N} assets")
        print(f"Expected returns range: [{mu.min():.4f}, {mu.max():.4f}]")
        print(f"Covariance range: [{sigma.min():.4f}, {sigma.max():.4f}]")
    
    # Step 2: Advanced partitioning with RMT
    if verbose:
        print(f"\nApplying RMT denoising and partitioning into {partition_size}-asset subproblems...")
    
    partitioned_mus, partitioned_sigmas, mappings = partition_with_rmt(
        sigma, mu, partition_size, n_obs
    )
    num_partitions = len(mappings)
    
    if verbose:
        print(f"Created {num_partitions} partitions")
        print(f"Partition sizes: {[len(m) for m in mappings]}")
    
    # Step 3: Distribute K across partitions
    K_per_partition = []
    remaining_K = K_total
    
    for i, mapping in enumerate(mappings):
        partition_size_actual = len(mapping)
        if i < num_partitions - 1:
            k_i = int(K_total * partition_size_actual / N)
            k_i = min(k_i, partition_size_actual)
            K_per_partition.append(k_i)
            remaining_K -= k_i
        else:
            K_per_partition.append(min(remaining_K, partition_size_actual))
    
    if verbose:
        print(f"K distribution across partitions: {K_per_partition}")
    
    # Step 4: Solve each subproblem with QAOA
    if verbose:
        print(f"\nSolving {num_partitions} subproblems with QAOA (p={p})...")
    
    sub_solutions = []
    sub_approximation_ratios = []
    total_start_time = time.time()
    
    for i in range(num_partitions):
        if verbose:
            print(f"\nPartition {i+1}/{num_partitions}:")
        
        # Create subproblem with risk rebalancing
        sub_problem = get_sub_problem_with_rebalancing(
            sigma=partitioned_sigmas[i],
            mu=partitioned_mus[i],
            K=K_per_partition[i],
            q=q,
            seed=1,
            scale=scale,
            full_sigma=sigma,
            full_mu=mu
        )
        
        # Solve with QAOA
        bitstring, opt_params, sub_ar = solve_qaoa_subproblem(sub_problem, p=p, verbose=verbose)
        sub_solutions.append(bitstring)
        sub_approximation_ratios.append(sub_ar)
        
        if verbose:
            print(f"  Solution: {bitstring} (selected {bitstring.count('1')} assets)")
    
    # Step 5: Map solutions back to full problem
    if verbose:
        print("\nMapping subproblem solutions to full solution...")
    
    full_solution = map_qaoa_outputs_to_full_vector(sub_solutions, mappings, N)
    
    total_time = time.time() - total_start_time
    
    # Step 6: Evaluate final solution
    selected_assets = np.where(full_solution == 1)[0]
    num_selected = len(selected_assets)
    
    if verbose:
        print(f"\nFinal solution:")
        print(f"  Total assets selected: {num_selected} (target: {K_total})")
        print(f"  Selected asset indices: {selected_assets.tolist()}")
        print(f"  Total computation time: {total_time:.2f} seconds")
    
    # Calculate portfolio metrics
    x = full_solution
    portfolio_return = np.dot(mu, x)
    portfolio_risk = np.dot(x, np.dot(sigma, x))
    objective_value = q * portfolio_risk - portfolio_return
    
    # Calculate final approximation ratio
    if verbose:
        print("\nCalculating final approximation ratio...")
    
    final_ar = calculate_final_approximation_ratio(x, sigma, mu, K_total, q)
    
    # Calculate average subproblem AR
    avg_sub_ar = np.mean(sub_approximation_ratios) if sub_approximation_ratios else 0
    
    if verbose:
        print(f"\nPortfolio metrics:")
        print(f"  Expected return: {portfolio_return:.6f}")
        print(f"  Risk (variance): {portfolio_risk:.6f}")
        print(f"  Objective value: {objective_value:.6f}")
        print(f"\nApproximation Ratios:")
        print(f"  Average subproblem AR: {avg_sub_ar:.6f}")
        print(f"  Final solution AR: {final_ar:.6f}")
        print(f"  {'='*50}")
        print(f"  FINAL AR: {final_ar:.3f}")
        print(f"  {'='*50}")
    
    return full_solution, selected_assets, {
        'return': portfolio_return,
        'risk': portfolio_risk,
        'objective': objective_value,
        'approximation_ratio': final_ar,
        'subproblem_approximation_ratios': sub_approximation_ratios,
        'average_subproblem_ar': avg_sub_ar,
        'computation_time': total_time,
        'num_partitions': num_partitions
    }


# Example usage
if __name__ == "__main__":
    # Configuration
    PICKLE_FILE = "subgraph_150.pickle"
    PARTITION_SIZE = 7  # Each partition will have ~15 assets
    Q = 0.5  # Risk aversion parameter
    K_TOTAL = 30  # Total number of assets to select
    P = 5  # QAOA depth
    SCALE = 1.0  # Scaling factor
    N_OBS = None  # Number of observations (5 years of daily data)
    
    # Run the advanced partitioned portfolio optimization
    solution, selected_assets, metrics = main(
        pickle_file=PICKLE_FILE,
        partition_size=PARTITION_SIZE,
        q=Q,
        K_total=K_TOTAL,
        p=P,
        scale=SCALE,
        n_obs=N_OBS,
        verbose=True
    )
    
    # Save results
    results = {
        'solution': solution,
        'selected_assets': selected_assets,
        'metrics': metrics,
        'parameters': {
            'partition_size': PARTITION_SIZE,
            'q': Q,
            'K_total': K_TOTAL,
            'p': P,
            'scale': SCALE,
            'n_obs': N_OBS
        }
    }
    
    # with open('advanced_partitioned_qaoa_results.pickle', 'wb') as f:
    #     pickle.dump(results, f)
    
    # print("\nResults saved to 'advanced_partitioned_qaoa_results.pickle'")